In [3]:
import cv2
import numpy as np
import mediapipe as mp
import logging
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [4]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle


In [5]:
# Initialize optical flow variables
prev_frame = None
prev_hip = None
prev_knee = None
prev_ankle = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("../test/CST_self2.mp4")
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            logging.warning("No frame captured from the video source.")
            break

        frame_height, frame_width, _ = frame.shape
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * frame_width, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * frame_height]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x * frame_width, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y * frame_height]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * frame_width, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * frame_height]

            # Draw optical flow displacement
            if prev_frame is not None:
                for prev_point, curr_point, color in zip([prev_hip, prev_knee, prev_ankle], [hip, knee, ankle], [(0, 0, 255)]*3):
                    prev_x, prev_y = map(int, prev_point)
                    curr_x, curr_y = map(int, curr_point)
                    cv2.circle(image, (curr_x, curr_y), 5, color, -1)
                    cv2.line(image, (prev_x, prev_y), (curr_x, curr_y), (0, 255, 0), 2)

            prev_frame = frame.copy()
            prev_hip, prev_knee, prev_ankle = hip, knee, ankle

        cv2.imshow("Mediapipe Feed", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    cap.release()

I0000 00:00:1716908704.886464 2345632 gl_context.cc:357] GL version: 2.1 (2.1 ATI-5.2.4), renderer: AMD Radeon Pro 5500M OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/Users/brennanlee/Desktop/opencv-healthcare/myenv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [6]:
# keypoints_history
keypoints
# displacements
# velocities

array([[[ 905.94904, 1431.8617 ]],

       [[ 596.29346, 1446.9176 ]],

       [[ 599.96277, 1754.2441 ]]], dtype=float32)